# Vertex AI GenAI For BigQuery Metadata - Make Better Tables

BigQuery tables are a great source of information for generative AI applications.  Retrieving information is a multi-step process as covered in [Vertex AI GenAI For BigQuery Q&A - Overview](./Vertex%20AI%20GenAI%20For%20BigQuery%20Q&A%20-%20Overview.ipynb).  The ability of a large language model to understand the contents of tables directly relies on the descriptiveness of the metadata: column names, column descriptions, table names, table descriptions.  

This workflow shows the potential for creating better, more descriptive metadata for BigQuery tables by using the existing metadata as well as common values from the tables columns.

This notebooks uses the BigFrames API for BigQuery to make local work in the form of a Pandas like API while keeping the execution remote, within BigQuery.  The LLM used here is Vertex AI [text-bison](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text) called directly from BigQuery using [ML.GENERATE_TEXT](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text) using the api [bigframes.ml.llm.PaLM2TextGenerator()](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.llm.PaLM2TextGenerator).


---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Vertex%20AI%20GenAI%20For%20BigQuery%20Metadata%20-%20Make%20Better%20Tables.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [ ]:
PROJECT_ID = 'demos-vertex-ai' # replace with project ID

In [ ]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment.  Also, the API for Artifact Registry needs to be enabled (if not already enabled).

### Installs (If Needed)
The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [ ]:
# tuples of (import name, install name)
packages = [
    ('bigframes', 'bigframes'),
    ('google.cloud.bigquery_connection_v1', 'google-cloud-bigquery-connection'),
    ('google.cloud.bigquery', 'google-cloud-bigquery')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user

In [ ]:
! pip install --upgrade bigframes -U -q --user

### API Enablement

Make sure the [BigQuery Connection API](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) is enabled:

In [ ]:
!gcloud services enable bigqueryconnection.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [ ]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'demos-vertex-ai'

In [2]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'bq-metadata'

In [3]:
# make this the BQ Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = EXPERIMENT
BQ_REGION = REGION[0:2] # subset to first two characters for multi-region

In [4]:
from IPython.display import Markdown
import bigframes.pandas as bf
import bigframes.ml as bfml
from bigframes.ml import llm
from bigframes.ml import ensemble
from google.cloud import bigquery_connection_v1 as bq_connection

In [5]:
bf.reset_session()
bf.options.bigquery.project = BQ_PROJECT
bf.options.bigquery.location = BQ_REGION
bf_session = bf.get_global_session()

---
## Review Data Source

The data source here is a product catalog with source:
- BigQuery Public table `bigquery-public-data.thelook_ecommerce.products`.



In [6]:
BQ_SOURCE = 'bigquery-public-data.thelook_ecommerce.products'

### Get Table: BigQuery Public Table

In [7]:
products = bf.read_gbq(BQ_SOURCE)

In [8]:
products.dtypes

id                                  Int64
cost                              Float64
category                  string[pyarrow]
name                      string[pyarrow]
brand                     string[pyarrow]
retail_price                      Float64
department                string[pyarrow]
sku                       string[pyarrow]
distribution_center_id              Int64
dtype: object

In [9]:
products.describe()

,id,cost,retail_price,distribution_center_id
count,29120.0,29120.0,29120.0,29120.0
mean,14560.5,28.481774,59.220164,4.982898
std,8406.364256,30.624681,65.888927,2.901153
min,1.0,0.0083,0.02,1.0
25%,7256.0,11.24484,24.0,2.0
50%,14448.0,19.72495,39.990002,5.0
75%,21874.0,34.43,69.949997,8.0
max,29120.0,557.151002,999.0,10.0


In [10]:
products.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13021,23.295341,Swim,Coolibar UPF 50+ Women's Stash Pocket Swim Shi...,Coolibar,49.990002,Women,83A46F92EEC62C8EE20F7F38514607D2,5
1,25133,11.1649,Socks,Bridgedale Men's Xhale Light Hiker Socks,Bridgedale,17.950001,Men,B7F0B1C1761095727BB5513DF24A3302,1
2,7352,19.751511,Skirts,Anne Klein Women's Classic Skirt,Anne Klein,43.990002,Women,AB73F542B6D60C4DE151800B8ABC0A6C,2
3,3647,16.480291,Dresses,Plus size Sexy Black Single Shoulder Mini Dress,eVogues Apparel,34.990002,Women,0D8080853A54F8985276B0130266A657,5
4,8167,88.953,Suits,LE SUIT Harvest Berry Jacket/Pant Suit,Le Suit,149.0,Women,FEB93033A7AB04EA442B08103EC019A4,7


### Get Table Info From BigQuery Information Schema: Columns

Retrieve the metadata for the table from Information Schema views like [INFORMATION_SCHEMA.COLUMN_FIELD_PATHS](https://cloud.google.com/bigquery/docs/information-schema-column-field-paths)

**NOTE** When `column_name` is not equal to `field_path` it is because the column is nested withing a RECORD (think array, or list) or STRUCT (think dictionary of key:value pairs).  This example does not have examples of these but could be extended to handle these as well.

In [11]:
products_columns = bf.read_gbq(f"""
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE TABLE_NAME = 'products'
        AND column_name = field_path
""")

In [12]:
products_columns

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,bigquery-public-data,thelook_ecommerce,products,cost,cost,FLOAT64,<NA>,NULL,<NA>
1,bigquery-public-data,thelook_ecommerce,products,retail_price,retail_price,FLOAT64,<NA>,NULL,<NA>
2,bigquery-public-data,thelook_ecommerce,products,sku,sku,STRING,<NA>,NULL,<NA>
3,bigquery-public-data,thelook_ecommerce,products,distribution_center_id,distribution_center_id,INT64,<NA>,NULL,<NA>
4,bigquery-public-data,thelook_ecommerce,products,name,name,STRING,<NA>,NULL,<NA>
5,bigquery-public-data,thelook_ecommerce,products,category,category,STRING,<NA>,NULL,<NA>
6,bigquery-public-data,thelook_ecommerce,products,brand,brand,STRING,<NA>,NULL,<NA>
7,bigquery-public-data,thelook_ecommerce,products,department,department,STRING,<NA>,NULL,<NA>
8,bigquery-public-data,thelook_ecommerce,products,id,id,INT64,<NA>,NULL,<NA>


### Get Table Info From BigQuery Information Schema: Table

Retrieve the metadata for the table from Information Schema views like [INFORMATION_SCHEMA.TABLE_OPTIONS](https://cloud.google.com/bigquery/docs/information-schema-table-options)

This view has one row for each option within each table.  Here, only the `OPTION_NAME = 'description'` is needed.

In [13]:
products_table = bf.read_gbq(f"""
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.INFORMATION_SCHEMA.TABLE_OPTIONS`
    WHERE TABLE_NAME = 'products'
        AND OPTION_NAME = 'description'
""")

In [14]:
products_table

,table_catalog,table_schema,table_name,option_name,option_type,option_value
0,bigquery-public-data,thelook_ecommerce,products,description,STRING,"""The Look fictitious e-commerce dataset - prod..."


### Get Values From Columns: Most common values as examples

Retrieve a sample of common values from each column to use as examples for an LLM to create names and descriptions.

Create syntax for query that will create a row per column with a sample of values from the column.

In [15]:
for c, col in enumerate(products_columns.column_name.unique().tolist()):
    if c == 0: 
        cte = f"""SELECT '{col}' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT({col}, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))"""
    else:
        cte += f"""\nUNION ALL\nSELECT '{col}' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT({col}, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))"""
print(cte)

SELECT 'cost' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(cost, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))
UNION ALL
SELECT 'retail_price' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(retail_price, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))
UNION ALL
SELECT 'sku' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(sku, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))
UNION ALL
SELECT 'distribution_center_id' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(distribution_center_id, 10) as osn FROM `bigquery-public-data.thelook_ecommerce.products`))
UNION ALL
SELECT 'name' AS column_name, STRING_AGG(CAST(value AS STRING)) as column_sample FROM UNNEST((SELECT APPROX_TOP_COUNT(name, 10) as osn FROM `b

In [16]:
products_sample = bf.read_gbq(cte)

In [17]:
products_sample

,column_name,column_sample
0,cost,"13.549999985843897,10.750000039115548,12.05000..."
1,department,"Women,Men"
2,retail_price,"25,29.989999771118164,19.989999771118164,39.99..."
3,category,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ..."
4,brand,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N..."
5,id,"29120,29119,29118,29117,29116,29115,29114,2911..."
6,sku,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06..."
7,distribution_center_id,"2,1,3,8,4,9,7,6,5,10"
8,name,Wrangler Men's Premium Performance Cowboy Cut ...


---
## BigQuery ML: Connect To Vertex AI LLMs with ML.GENERATE_TEXT

BigQuery ML can `Create Model`s that are actually connections to Remote Models. [Reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model)

Using the `REMOTE_SERVICE_TYPE = "CLOUD_AI_LARGE_LANGUAGE_MODEL_V1"` option will link to LLMs in Vertex AI!

### Connection Requirement

To make a remote connection using BigQuery ML, BigQuery uses a CLOUD_RESOURCE connection. [Reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model#connection)

Create a new connection with type `CLOUD_RESOURCE`: First, check for existing connection.

In [18]:
try:
    response = bq_connection.ConnectionServiceClient().get_connection(
            request = bq_connection.GetConnectionRequest(
                name = f"projects/{BQ_PROJECT}/locations/{BQ_REGION}/connections/{SERIES}_{EXPERIMENT}"
            )
    )
    print(f'Found existing connection with service account: {response.cloud_resource.service_account_id}')
    service_account = response.cloud_resource.service_account_id
except Exception:
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": f"projects/{BQ_PROJECT}/locations/{BQ_REGION}",
            "connection_id": f"{SERIES}_{EXPERIMENT}",
            "connection": bq_connection.types.Connection(
                {
                    "friendly_name": f"{SERIES}_{EXPERIMENT}",
                    "cloud_resource": bq_connection.CloudResourceProperties({})
                }
            )
        }
    )
    response = bq_connection.ConnectionServiceClient().create_connection(request)
    print(f'Created new connection with service account: {response.cloud_resource.service_account_id}')
    service_account = response.cloud_resource.service_account_id
    # assign the service account the Vertex AI User Role:
    !gcloud projects add-iam-policy-binding {BQ_PROJECT} --member=serviceAccount:{service_account} --role=roles/aiplatform.user

Found existing connection with service account: bqcx-746038361521-dl2h@gcp-sa-bigquery-condel.iam.gserviceaccount.com


**NOTE**: The step above created a service account and assigned it the Vertex AI User Role.  This may take a moment to be recognized in the steps below.  If you get an error in one of the cells below try rerunning it.

### Create The Remote Model In BigQuery

Create a temp model that connects to text generation model on Vertex AI - [Reference](https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.ml.llm.PaLM2TextGenerator)

https://cloud.google.com/bigquery/docs/use-bigquery-dataframes#large-language-models


In [19]:
# textgen_model = bfml.llm.PaLM2TextGenerator(
#     session = bf_session,
#     connection_name = f'{BQ_PROJECT}.{BQ_REGION}.{SERIES}_{EXPERIMENT}',
#     model_name = "text-bison-32k"
# )

In [20]:
textgen_model = bfml.llm.GeminiTextGenerator(
    session = bf_session,
    connection_name = f'{BQ_PROJECT}.{BQ_REGION}.{SERIES}_{EXPERIMENT}'
)

---
## Generate Table Metadata

### Bring Together Column Information

In [21]:
products_columns.columns

Index(['table_catalog', 'table_schema', 'table_name', 'column_name',
       'field_path', 'data_type', 'description', 'collation_name',
       'rounding_mode'],
      dtype='object')

In [22]:
products_sample.columns

Index(['column_name', 'column_sample'], dtype='object')

In [23]:
products_columns = products_columns[['column_name', 'data_type', 'description']].merge(products_sample, on = 'column_name')

In [24]:
products_columns

,column_name,data_type,description,column_sample
0,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000..."
1,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99..."
2,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06..."
3,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10"
4,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...
5,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ..."
6,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N..."
7,department,STRING,<NA>,"Women,Men"
8,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911..."


### Add Table Information

In [25]:
products_columns['table_name'] = products_table['table_name'].iloc[0]
products_columns['table_description'] = products_table['option_value'].iloc[0]

In [26]:
products_columns

,column_name,data_type,description,column_sample,table_name,table_description
0,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000...",products,"""The Look fictitious e-commerce dataset - prod..."
1,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99...",products,"""The Look fictitious e-commerce dataset - prod..."
2,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06...",products,"""The Look fictitious e-commerce dataset - prod..."
3,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10",products,"""The Look fictitious e-commerce dataset - prod..."
4,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...,products,"""The Look fictitious e-commerce dataset - prod..."
5,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ...",products,"""The Look fictitious e-commerce dataset - prod..."
6,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N...",products,"""The Look fictitious e-commerce dataset - prod..."
7,department,STRING,<NA>,"Women,Men",products,"""The Look fictitious e-commerce dataset - prod..."
8,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911...",products,"""The Look fictitious e-commerce dataset - prod..."


### Create Column Naming Prompt

In [27]:
products_columns['name_prompt'] = (
    'Generate a new column name for a BigQuery column with the following information. '
    + 'The current column name is ' + products_columns['column_name'] + '. '
    + 'The table has the name ' + products_columns['table_name'] + '. '
    + 'The column has a datatype of ' + products_columns['data_type'] + ' with common values like: ' + products_columns['column_sample'] + '.'
)

In [28]:
products_columns

,column_name,data_type,description,column_sample,table_name,table_description,name_prompt
0,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
1,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
2,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
3,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
4,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...,products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
5,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
6,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
7,department,STRING,<NA>,"Women,Men",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...
8,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...


In [29]:
products_columns['name_prompt'].iloc[2]

'Generate a new column name for a BigQuery column with the following information. The current column name is sku. The table has the name products. The column has a datatype of STRING with common values like: FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06D87E6BA6C830F3B6284,FFF75F52998A477F6E7B00E58AF8D64A,FFF6F444C0FD08F8CCA7EB254A216565,FFF574293A6252F4029A9413F364B2E6,FFF38493F5A1643EE8EF247750540EE2,FFF079091FAB64095F9CAD3298F1057A,FFEED84C7CB1AE7BF4EC4BD78275BB98,FFEDF5BE3A86E2EE281D54CDC97BC1CF,FFEC51567543679F01CE65724ADCA743.'

### Generate New Column Names

In [30]:
products_columns = products_columns.join(textgen_model.predict(products_columns['name_prompt']).rename(columns={'ml_generate_text_llm_result':'new_column_name'}))

In [31]:
products_columns

,column_name,data_type,description,column_sample,table_name,table_description,name_prompt,new_column_name,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_cost_float64,null,,Generate a new column name for a BigQuery colu...
1,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,unit_price,null,,Generate a new column name for a BigQuery colu...
2,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,`products_sku`,null,,Generate a new column name for a BigQuery colu...
3,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,dc_id,null,,Generate a new column name for a BigQuery colu...
4,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...,products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_name,null,,Generate a new column name for a BigQuery colu...
5,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_category,null,,Generate a new column name for a BigQuery colu...
6,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,brand_name,null,,Generate a new column name for a BigQuery colu...
7,department,STRING,<NA>,"Women,Men",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_department_gender,null,,Generate a new column name for a BigQuery colu...
8,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,productId,null,,Generate a new column name for a BigQuery colu...


### TODO ========

In [32]:
list(products_columns.columns)

['column_name',
 'data_type',
 'description',
 'column_sample',
 'table_name',
 'table_description',
 'name_prompt',
 'new_column_name',
 'ml_generate_text_rai_result',
 'ml_generate_text_status',
 'prompt']

In [33]:
type(products_columns)

bigframes.dataframe.DataFrame

In [34]:
# Save the DataFrame to BigQuery for further analysis outside this notebook
products_columns.to_gbq(
    destination_table = "bq_metadata.products_columns_check",
    # project_id = PROJECT_ID,
    if_exists = "replace",
)  

'bq_metadata.products_columns_check'

In [ ]:
# example = products_columns["ml_generate_text_rai_result"].head(1)

In [37]:
textgen_model.predict(
    'The context for a BigQuery table column follows. '
    + 'The column name is ' + products_columns['new_column_name'] + '. '
    + 'The table has the name ' + products_columns['table_name'] + '. '
    + 'The column has a datatype of ' + products_columns['data_type'] + ' with common values like: ' + products_columns['column_sample'] + '. '
    + 'Generate a description the column.'
)

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,The products_cost_float64 column in the produc...,null,,The context for a BigQuery table column follow...
1,The unit_price column in the products table re...,null,,The context for a BigQuery table column follow...
2,The `products_sku` column in the `products` ta...,null,,The context for a BigQuery table column follow...
3,**Column Name** dc_id **Table Name** products...,null,,The context for a BigQuery table column follow...
4,The `product_name` column in the `products` ta...,null,,The context for a BigQuery table column follow...
5,The product_category column in the products ta...,null,,The context for a BigQuery table column follow...
6,The brand_name column in the products table co...,null,,The context for a BigQuery table column follow...
7,The `products_department_gender` column in the...,null,,The context for a BigQuery table column follow...
8,The productId column in the products table sto...,null,,The context for a BigQuery table column follow...


### Generate New Column Description

In [ ]:
# products_columns = products_columns.drop(columns = 'new_column_description')

In [35]:
products_columns = products_columns.join(textgen_model.predict(
    'The context for a BigQuery table column follows. '
    + 'The column name is ' + products_columns['new_column_name'] + '. '
    + 'The table has the name ' + products_columns['table_name'] + '. '
    + 'The column has a datatype of ' + products_columns['data_type'] + ' with common values like: ' + products_columns['column_sample'] + '. '
    + 'Generate a description the column.'
).rename(columns={'ml_generate_text_llm_result':'new_column_description'}))

NotImplementedError: Deduping column names is not implemented. Share your usecase with the BigQuery DataFrames team at the https://bit.ly/bigframes-feedback survey.

In [36]:
products_columns

,column_name,data_type,description,column_sample,table_name,table_description,name_prompt,new_column_name,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,cost,FLOAT64,<NA>,"13.549999985843897,10.750000039115548,12.05000...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_cost_float64,null,,Generate a new column name for a BigQuery colu...
1,retail_price,FLOAT64,<NA>,"25,29.989999771118164,19.989999771118164,39.99...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,unit_price,null,,Generate a new column name for a BigQuery colu...
2,sku,STRING,<NA>,"FFFCC1A3964B4AD665FA2F07D7BFD086,FFFB8EF15DE06...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,`products_sku`,null,,Generate a new column name for a BigQuery colu...
3,distribution_center_id,INT64,<NA>,"2,1,3,8,4,9,7,6,5,10",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,dc_id,null,,Generate a new column name for a BigQuery colu...
4,name,STRING,<NA>,Wrangler Men's Premium Performance Cowboy Cut ...,products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_name,null,,Generate a new column name for a BigQuery colu...
5,category,STRING,<NA>,"Intimates,Jeans,Tops & Tees,Fashion Hoodies & ...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,product_category,null,,Generate a new column name for a BigQuery colu...
6,brand,STRING,<NA>,"Allegra K,Calvin Klein,Carhartt,Hanes,Volcom,N...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,brand_name,null,,Generate a new column name for a BigQuery colu...
7,department,STRING,<NA>,"Women,Men",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,products_department_gender,null,,Generate a new column name for a BigQuery colu...
8,id,INT64,<NA>,"29120,29119,29118,29117,29116,29115,29114,2911...",products,"""The Look fictitious e-commerce dataset - prod...",Generate a new column name for a BigQuery colu...,productId,null,,Generate a new column name for a BigQuery colu...


### Generate New Table Description

Convert selected column for schema into a markdown table for including in the prompt:

In [ ]:
markdown_products_columns = products_columns[['new_column_name', 'new_column_description', 'data_type']].rename(columns = {'new_column_name':'column_name', 'new_column_description':'description'}).to_pandas().to_markdown(index = False)

Review the markdown table:

In [ ]:
Markdown(markdown_products_columns)

Generate the table description:

In [ ]:
table_description = textgen_model.predict(bf.DataFrame({ "prompt": [
f"""Generate a description for the BigQuery table with schema:
{markdown_products_columns}
"""
],})).ml_generate_text_llm_result.iloc[0]

Markdown(table_description)

---
## Update Metadata For A Table

To show how to do an inplace update of column names and descriptions for tables and columns, the source table will be copied to a new dataset first.


**NOTE: SWITCHING TO BIGQUERY API**

Some of the tasks in this section require using feature of BigQuery DDL to alter tables that are not yet available in the BigFrames API.

### Setup BigQuery Client

In [ ]:
from google.cloud import bigquery
# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

### Create BigQuery Dataset

In [ ]:
# create/link to dataset
ds = bigquery.DatasetReference(BQ_PROJECT, BQ_DATASET)
ds.location = BQ_REGION
ds.labels = {'series': f'{SERIES}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True) 

### Copy Source Table To User Dataset

In [ ]:
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
SELECT *
FROM `{BQ_SOURCE}`
"""
print(query)

In [ ]:
job = bq.query(query = query)
job.result()
job.state

### Review Schema Before Update

In [ ]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE TABLE_NAME = '{BQ_TABLE}'
        AND column_name = field_path
"""
bq.query(query = query).to_dataframe()

In [ ]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.TABLE_OPTIONS`
    WHERE TABLE_NAME = '{BQ_TABLE}'
        AND OPTION_NAME = 'description'
"""
bq.query(query = query).to_dataframe()

### Change Column Names

Construct DDL to Alter Table:

In [ ]:
for r, row in enumerate(products_columns[['column_name', 'new_column_name', 'new_column_description']].values.tolist()):
    if r == 0:
        query = f"ALTER TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`"
        query += f"""\n\tRENAME COLUMN {row[0]} TO {row[1]}"""
    else:
        query += f""",\n\tRENAME COLUMN {row[0]} TO {row[1]}"""
print(query)

In [ ]:
job = bq.query(query = query)
job.result()
job.state

### Change Column Descriptions

Construct DDL to Alter Table:

In [ ]:
for r, row in enumerate(products_columns[['column_name', 'new_column_name', 'new_column_description']].values.tolist()):
    if r == 0:
        query = f"ALTER TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`"
        query += f"""\n\tALTER COLUMN {row[1]} SET OPTIONS (description = "{row[2].replace('"', "'")}")"""
    else:
        query += f""",\n\tALTER COLUMN {row[1]} SET OPTIONS (description = "{row[2].replace('"', "'")}")"""
print(query)

In [ ]:
job = bq.query(query = query)
job.result()
job.state

### Change Table Description

In [ ]:
query = f"""ALTER TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
\tSET OPTIONS(description = "{table_description.replace('"', "'")}")"""
print(query)

In [ ]:
job = bq.query(query = query)
job.result()
job.state

### Review Schema After Update

In [ ]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE TABLE_NAME = '{BQ_TABLE}'
        AND column_name = field_path
"""
bq.query(query = query).to_dataframe()

In [ ]:
query = f"""
    SELECT *
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.TABLE_OPTIONS`
    WHERE TABLE_NAME = '{BQ_TABLE}'
        AND OPTION_NAME = 'description'
"""
bq.query(query = query).to_dataframe()['option_value'].iloc[0]